# Example 1: Convert MS Access databases to csv

In this Notebook we will convert MS Access databases to csv in order to read them in our Notebook environment.

![axxess](img/axxess.png)

First we will prepare our workspace and download some Access databases. We will also download the [Axxess](https://github.com/DANS-repo/axxess) jar with dependencies that will extract metadata and data from the databases. We will inspect the output csv files, the database metadata and finally get our hands wet on the data.

### Prerequisites

- System requirement: java 8 or higher

In [1]:
!java -version

java version "1.8.0_101"
Java(TM) SE Runtime Environment (build 1.8.0_101-b13)
Java HotSpot(TM) 64-Bit Server VM (build 25.101-b13, mixed mode)


### Table of contents

- [Preparation of the work space](#pows)
    * [Create work directories](#pows_work)
    * [Example Access files](#pows_exam)
    * [Download Axxess jar-with-dependencies](#pow_jar)
    * [Access help](#pow_help)
 
  
- [Access to csv](#aca)
    * [Write a properties file](#acc_prop)
    * [Start Axxess...](#acc_start)
    * [Inspect the output](#acc_out)
    * [Inspect the database metadata](#acc_dbmd)
    * [Work with the actual data](#acc_work)

# Preparation of the work space <a id="pows"></a>

### Create work directories <a id="pows_work"></a>

Create directories were we can store source data files and the Axxess jar-file:

In [2]:
import os

os.makedirs('java', exist_ok=True)
os.makedirs('work/src-data', exist_ok=True)

### Example Access files <a id="pows_exam"></a>

In order to get a live experience you need to download some MS Access files or provide your own. Here are some nice examples from the [European Environment Agency](https://www.eea.europa.eu/). They have `csv` files of the same stuff, but that's not the point. We want the `.mdb` files to get this little demonstration going. You have to download the files manually, because they want you to give a one-question-only feedback and you need to unzip the files. Save the `.mdb` files in our data source directory `work/src-data`.

- [European Red Lists of species](https://www.eea.europa.eu/ds_resolveuid/df20c2cbd12e4f1299c16ea8bf78a5c2)
- [Article 17, Habitats Directive](https://www.eea.europa.eu/ds_resolveuid/78a8fdf22fa14fddb8ff218071aeb5d8)

If the above links don't work anymore, that's fine. Just try and get some other MS Access databases.

In [3]:
!ls -la work/src-data

total 358584
drwxr-xr-x  4 ecco  staff        128 Mar 16 21:54 .
drwxr-xr-x  5 ecco  staff        160 Mar 16 22:47 ..
-rwxr-xr-x@ 1 ecco  staff  130617344 Sep  2  2015 Art17_MS_EU27_2015.mdb
-rwxr-xr-x@ 1 ecco  staff   52977664 Nov 23 16:24 European_Red_List_November2017.mdb


### Download the Axxess jar-with-dependencies <a id="pow_jar"></a>

Here we get the Axxess-jar-with-dependencies.jar from [GitHub](https://github.com/DANS-repo/axxess) - of course we only need to do this once. Adjust the version number (`1.0.1`) in the url if needed.

In [4]:
import requests

def download_file(url, filename):
    mb = 2**20
    bts = 0
    print("Downloading", url)
    r = requests.get(url, stream=True)
    with open(filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            bts += 1024
            if chunk:
                f.write(chunk)
                if bts % mb == 0:
                    print('.', end='', flush=True)
    return filename, bts, bts/mb

In [5]:
url = "https://github.com/DANS-repo/axxess/releases/download/1.0.1/axxess-jar-with-dependencies.jar"
filename = "java/axxess-jar-with-dependencies.jar"
download_file(url, filename)

.........................................................

('java/axxess-jar-with-dependencies.jar', 60484608, 57.6826171875)

### Access help <a id="pow_help"></a>

Access help will briefly inform you about its usage. Help will be printed if invoking axxess with no arguments or with arguments `-h` or `--help`.

In [6]:
!java -jar java/axxess-jar-with-dependencies.jar

No log configuration given. Using default. See: logs/axxess.log
Axxess is a tool for converting MS Access databases to and from csv files.
See also: https://github.com/DANS-repo/axxess

USAGE:

          java -jar axxess-jar-with-dependencies.jar [axxess.properties] [logback configuration]

axxess.properties      - configuration file.
                         See https://github.com/DANS-repo/axxess/blob/master/docker/cfg/axxess.properties
                         If no properties file given will look for cfg/axxess.properties

logback configuration  - logging configuration.
                         See https://logback.qos.ch/manual/configuration.html
                         If no logging configuration given will log to logs/axxess.log


# Access to csv <a id="aca"></a>

Axxess can be configured and instructed with a properties file. An example properties file and documentation can be found [here](https://github.com/DANS-repo/axxess/blob/master/docker/cfg/axxess.properties). Most properties have a default value, the only ones you need to provide for Access to csv conversion are `axxess.mode` and `db.source.file`.

```
# ===================================
# axxess.mode = aca | ac | ca
# There are 3 modes:
#
# 1. (aca) convert access databases to csv and the csv again to an access database(s) (in another version).
# 2. (ac)  convert access databases to csv.
# 3. (ca)  convert csv to access database(s).
```

```
# ===================================
# db.source.file = {directory} | {file}
#
# if given a directory will walk directory structure recursively and convert all access databases found.
# if given a file and the file points to an access database wil convert this file.
```

Let's create a properties file and convert our example databases in `work/src-data` to csv.

### Write a properties file <a id="acc_prop"></a>

In [7]:
%%writefile java/axxess.properties
axxess.mode=ac
db.source.file=work/src-data
csv.target.directory=work/out-csv
csv.result.list.file=work/acc2csv.txt

Writing java/axxess.properties


Notice that we gave a third property: `csv.target.directory`. This tells Axxess to output the csv files in our `work` directory. The subdirectory `out-csv` will be created by Axxess. Default output directory is `work/axxess-csv-out`.

Notice we also gave the property `csv.result.list.file`. This tells Axxess to create a simple list of all the newly created output files in the file `work/acc2csv.txt`. This overview may come in handy when working with the csv files. 

### Start Axxess... <a id="acc_start"></a>

In [8]:
!java -jar java/axxess-jar-with-dependencies.jar java/axxess.properties

No log configuration given. Using default. See: logs/axxess.log
Converted 2 database(s) to 55 result files, with 0 error(s) and 0 warnings(s).
See logs for details


We have successfully converted all the databases in `work/src-data` to csv files in `work/out-csv`.

### Inspect the output <a id="acc_out"></a>

In [9]:
!ls -la work/out-csv

total 0
drwxr-xr-x   4 ecco  staff   128 Mar 16 22:48 .
drwxr-xr-x   7 ecco  staff   224 Mar 16 22:48 ..
drwxr-xr-x  40 ecco  staff  1280 Mar 16 22:48 Art17_MS_EU27_2015_mdb
drwxr-xr-x  19 ecco  staff   608 Mar 16 22:48 European_Red_List_November2017_mdb


It has 2 directories with the name of the original databases. The dot (.) between filename and extension has been replaced with an underscore (\_) in the directory names. If our databases had been placed in a deeper directory structure, that structure would have been mimicked in our output directory. If in our property file we would have stated that target directory (`csv.target.directory`) be the same as source directory (`db.source.file`) the directories with csv files would have been created along side the original databases.

Let's inspect the output in `European_Red_List_November2017_mdb`...

In [10]:
!ls -la work/out-csv/European_Red_List_November2017_mdb

total 58600
drwxr-xr-x  19 ecco  staff      608 Mar 16 22:48 .
drwxr-xr-x   4 ecco  staff      128 Mar 16 22:48 ..
-rw-r--r--   1 ecco  staff   567111 Mar 16 22:48 European_Red_List_November2017.mdb.European_Amphibians_Reptiles_Red_List_Nov_2009.csv
-rw-r--r--   1 ecco  staff  3842263 Mar 16 22:48 European_Red_List_November2017.mdb.European_Bees_Red_List_April_2015.csv
-rw-r--r--   1 ecco  staff  2075694 Mar 16 22:48 European_Red_List_November2017.mdb.European_Birds_Red_List_June_2015.csv
-rw-r--r--   1 ecco  staff  1050113 Mar 16 22:48 European_Red_List_November2017.mdb.European_Butterflies_Red_List_March_2010.csv
-rw-r--r--   1 ecco  staff   326659 Mar 16 22:48 European_Red_List_November2017.mdb.European_Dragonflies_Red_List_March_2010.csv
-rw-r--r--   1 ecco  staff   744036 Mar 16 22:48 European_Red_List_November2017.mdb.European_Freshwater_Fishes_Red_List_Sep_2011.csv
-rw-r--r--   1 ecco  staff  1790905 Mar 16 22:48 European_Red_List_November2017.mdb.European_Freshwater_Molluscs_Re

All the tables from the original database are converted to separate csv files. The name of the csv files is a composition of
```
{filename of the original database}.{table name}.csv
```
Besides the tables there are two other files, `manifest-sha1.txt`, a text file containing the sha1 checksums of all the csv files and `European_Red_List_November2017.mdb._metadata.csv`, a metadata file with database properties, relationships, indexes, table and column properties and queries found in the database. Let's inspect the metadata.

### Inspect the database metadata <a id="acc_dbmd"></a>

In [11]:
import pandas as pd
pd.options.display.max_colwidth = 0

df = pd.read_csv("work/out-csv/Art17_MS_EU27_2015_mdb/Art17_MS_EU27_2015.mdb._metadata.csv")
df.head()

,Obj,Key,Type,Value
0,[EM],Conversion date,TEXT,2018-03-16T21:48:37.367Z
1,[EM],OS name,TEXT,Mac OS X
2,[EM],OS arch,TEXT,x86_64
3,[EM],OS version,TEXT,10.13.3
4,[EM],Axxess version,TEXT,1.0.1


The database metadata is a table, four columns wide: `Obj` (for Object), `Key`, `Type` and `Value`. See the javadoc on (Jackcess Enum) [Datatype](http://jackcess.sourceforge.net/apidocs/com/healthmarketscience/jackcess/DataType.html) if you want to know more about the values in column `Type`. The first `Obj` in the rows depicted above is `[EM]`. Not exactly an an object from the original database. `EM` stands for Extraction Metadata and gives us some provenance on the conversion we just caried out. Other values you may encounter in the `Obj` column:

- `[EM]` - as said before:  Extraction Metadata 
- `[DB]` - Database properties
- `[R0]` - Relationship metadata. The digit is an index number `[R0], [R1], [R2]... [Rn]` 
- `[Q0]` - Query, `[Q0], [Q1]... [Qn]`
- `[X0]` - Index metadata, `[X0], [X1]... [Xn]`
- `[T0]` - Table properties, `[T0], [T1]... [Tn]`
- `[T0][X0]` - Table index properties
- `[T0][C0]` - Table column properties

#### Example database metadata: Get column properties

List properties of column 5 in table 1...

In [12]:
df[df.Obj=='[T1][C5]']

,Obj,Key,Type,Value
660,[T1][C5],Column name,TEXT,location_number
661,[T1][C5],Column index,INT,5
662,[T1][C5],Data type,TEXT,TEXT
663,[T1][C5],Length,INT,510
664,[T1][C5],Length in units,INT,255
665,[T1][C5],Scale,BYTE,0
666,[T1][C5],Precision,BYTE,0
667,[T1][C5],IsAppendOnly,BOOLEAN,false
668,[T1][C5],IsAutoNumber,BOOLEAN,false
669,[T1][C5],IsCalculated,BOOLEAN,false


#### Example database metadata: List all columns from a table

To get name, data type and description of all columns in table 2...

In [13]:
crit = "\\[T2\\]\\[C[0-9]*\\]" # regex to select all columns in T2
names = df[(df.Obj.str.match(crit)) & (df.Key=="Column name")][['Obj', 'Value']]
types = df[(df.Obj.str.match(crit)) & (df.Key=="Data type")][['Obj', 'Value']]
descr = df[(df.Obj.str.match(crit)) & (df.Key=="(Property)Description")][['Obj', 'Value']]
dft = names.merge(types, left_on='Obj', right_on='Obj', how='outer') \
    .merge(descr, left_on='Obj', right_on='Obj', how='outer') \
    .fillna('')
dft.columns = ['column', 'name', 'data type', 'description']
dft

,column,name,data type,description
0,[T2][C0],greporthash,TEXT,General report auto generated primary key
1,[T2][C1],country,TEXT,2-letter ISO country code
2,[T2][C2],achievements,MEMO,Main achievements under the Habitats Directive (in national language)
3,[T2][C3],achievements_trans,MEMO,Main achievements under the Habitats Directive (in English)
4,[T2][C4],general_information,MEMO,General information on the Habitats Directive
5,[T2][C5],information_on_network,MEMO,Information on the Natura 2000 network in the Member State
6,[T2][C6],monitoring_schemes,MEMO,
7,[T2][C7],protection_of_species,MEMO,
8,[T2][C8],transpose_directive,MEMO,
9,[T2][C9],sites_total_number,INT,surface area in km2


## Work with actual data <a id="acc_work"></a>

Of course, the reason you converted Access to csv was to get grip on the data in these databases. Well go ahead...

In [14]:
pd.reset_option('max_colwidth')
dfm = pd.read_csv("work/out-csv/Art17_MS_EU27_2015_mdb/Art17_MS_EU27_2015.mdb.data_gmeasures.csv")
dfm.head()

,greporthash,sitecode,sitename,project_year,project_title,commission_opinion,project_impact
0,e9026d569bfc02c8236dc3dd61b7bad3,ES6200004,Sierras y vega alta del Segura y ríos Alhárabe...,2010,Sistema de suministro desde el embalse del Cen...,False,Detracción de caudales. Destrucción de tipos d...
1,e9026d569bfc02c8236dc3dd61b7bad3,ES0000235,De S&Albufera a la Mola,2010,Ampliación del vertedero de residuos no peligr...,False,Afección a tipos de hábitats de interés comuni...
2,e9026d569bfc02c8236dc3dd61b7bad3,ES0000337,Estrecho,2007,Segundo circuito de interconexión eléctrica Es...,False,Afección a tipos de hábitats de interés comuni...
3,e9026d569bfc02c8236dc3dd61b7bad3,ES0000210,Alto Sil,2012,Plan regional de ámbito sectorial de explotaci...,True,Afección a tipos de hábitats deinterés comunit...
4,13e8f2e8618fd0cc59d8d629d4e85579,BE2500001,Duinengebied inclusief IJzermonding en Zwin’,2007,Werken plan Oostende – in het kader van Master...,False,"Impact on sand banks, young dune and silt rela..."
